# Homework 2 - word count #

In [1]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('Word Count').setMaster('local')
sc = SparkContext(conf=conf)

docs = sc.textFile('dataset.txt', 4).persist()  #returns an RDD (there are at least 4 partitions)
print("Number of documents: ", docs.count())

Number of documents:  5


## Naive version ##

In [2]:
words = docs.flatMap(lambda doc: doc.split())
print("Number of words: ", words.count())

couples = words.map(lambda word: (word, 1))\
                .reduceByKey(lambda a,b: a+b)
print("Number of different words: ", couples.count())

Number of words:  56
Number of different words:  32


## Improved version 1 ##

In [5]:
def wordcount(document):
    dictionary = {}  #create empty dictionary to contain (string: integer) couples
    for word in document.split():  #word is a string element from the iterable list given by split()
        if word in dictionary.keys():  #increase the number of occurences
            dictionary[word] = dictionary[word] + 1
        else:
            dictionary[word] = 1
    return [(k,dictionary[k]) for k in dictionary.keys()]  #return a list of couples (string: integer)
                                                           #because returning the dictionary makes Spark angry

couples1 = docs.flatMap(wordcount)\
                .reduceByKey(lambda a,b: a+b)

In [6]:
print("Number of different couples: ", couples1.count())

Number of different couples:  32
